<a href="https://colab.research.google.com/github/sanket111219/galytix/blob/main/DataGalytix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path_x="/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz"

In [3]:
import gensim
from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format(path_x, binary=True, limit=1000000)
wv.save_word2vec_format('vectors.csv')

In [7]:
import pandas as pd


phrases_df = pd.read_csv('/content/drive/MyDrive/DE assessment/phrases.csv', encoding= 'unicode_escape')


def get_phrase_embeddings(phrase, wv):
    tokens = phrase.split()
    phrase_vec = [wv[word] for word in tokens if word in wv.key_to_index]
    return phrase_vec


phrases_df['embeddings'] = phrases_df['Phrases'].apply(lambda x: get_phrase_embeddings(x, wv))


In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


def calculate_similarity_matrix(phrases_df):
    embeddings = phrases_df['embeddings'].values
    num_phrases = len(embeddings)
    similarity_matrix = np.zeros((num_phrases, num_phrases))

    for i in range(num_phrases):
        for j in range(num_phrases):
            if i == j:
                similarity_matrix[i, j] = 1.0
            else:
                if embeddings[i] and embeddings[j]:
                    similarity_matrix[i, j] = cosine_similarity(embeddings[i], embeddings[j])[0][0]

    return similarity_matrix


similarity_matrix = calculate_similarity_matrix(phrases_df)




In [10]:
def find_closest_match(user_input, phrases_df):
    user_input_embedding = get_phrase_embeddings(user_input, wv)

    if user_input_embedding:
        similarities = []
        for phrase, embeddings in zip(phrases_df['Phrases'], phrases_df['embeddings']):
            if embeddings:
                similarity = cosine_similarity(user_input_embedding, embeddings)[0][0]
                similarities.append((phrase, similarity))

        closest_match = max(similarities, key=lambda x: x[1])
        return closest_match
    else:
        return "No valid embeddings found for the input phrase."


user_input_phrase = "How many passengers were there in titanic"
closest_match = find_closest_match(user_input_phrase, phrases_df)
print("Closest Match:", closest_match[0])
print("Cosine Similarity:", closest_match[1])


Closest Match: How is the airline industry performing globally?
Cosine Similarity: 1.0000002
